In [39]:
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import os
import shutil

In [3]:
def average_BGR(image):
    avg_blue = image[:,:,0].mean()
    avg_green = image[:,:,1].mean()
    avg_red = image[:,:,2].mean()

    return [avg_blue,avg_green,avg_red]

def average_HSV(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    avg_hue = image[:,:,0].mean()
    avg_saturation = image[:,:,1].mean()
    avg_value = image[:,:,2].mean()

    return avg_hue, avg_saturation, avg_value

In [4]:
list_avg_blue = []
list_avg_green = []
list_avg_red = []

list_avg_hue=[]
list_avg_saturation=[]
list_avg_value=[]
X1 = []
X2 = []
list_image = []
list_image_path=[]
for dirname, _, filenames in os.walk('D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix'):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        list_image_path.append(image_path)
        image = cv2.imread(image_path)
        list_image.append(image)
        avg_BGR = average_BGR(image)
        X1.append(avg_BGR)
        list_avg_blue.append(avg_BGR[0])
        list_avg_green.append(avg_BGR[1])
        list_avg_red.append(avg_BGR[2])
        avg_HSV= average_HSV(image)
        list_avg_hue.append(avg_HSV[0])
        list_avg_saturation.append(avg_HSV[1])
        list_avg_value.append(avg_HSV[2])
        X2.append(avg_HSV)

In [69]:
from sklearn.cluster import KMeans
kmeans1 = KMeans(n_clusters=10,random_state =0)

X1 = np.array(X1)

kmeans1.fit(X1)


print('Centers found by scikit-learn:')
# print(kmeans1.cluster_centers_)
pred_label = kmeans1.predict(X1)
# print(pred_label)

# for i in range(10):
#     folder_name = str(i)
#     os.mkdir(os.path.join('D:\PTIT\CSDLDPT\Multimedia-Database-System\kmeans1', folder_name))

dict = {}

for i in range(len(list_image_path)):
    dict[list_image_path[i]] = pred_label[i]

for key, value in dict.items():
    for dirname, child_folders, filenames in os.walk('kmeans1'):
        for folder in child_folders:
            if str(value) == folder:
                shutil.copy(key, os.path.join(dirname, folder))

for key, value in dict.items():
    if(value == 1):
        print(key, ':', value)







Centers found by scikit-learn:
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (01).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (05).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (06).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Avocado (10).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Eggplant (01).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (01).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (03).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (04).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (05).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (08).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mango (10).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mulberry (01).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mulberry (02).jpg : 1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Mulberry (03).jpg : 1
D:\PTIT\CSDLDPT\Multimedi

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
kmeans2 = KMeans(n_clusters=11,random_state =0)

X2 = np.array(X2)

kmeans2.fit(X2)

print('Centers found by scikit-learn:')
print(kmeans2.cluster_centers_)

Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 23.99286786  97.65680714 191.39643571]
 [ 94.85196     94.31786    107.31125333]
 [ 20.26515333 172.23953333 173.90971333]
 [ 21.74945769  69.93094231 195.87871154]
 [ 46.78221935  55.6506129  155.11283548]
 [ 39.788485   127.585285   100.630485  ]
 [ 27.44935714  26.31848571 179.77884286]
 [ 17.480295    53.495255   222.478025  ]
 [ 22.03905926  69.8302037  144.14539259]
 [ 21.75761429  96.81638571 164.37935714]]


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
result = []
result_pred_label2 = []
for i in range(11):
  len_label = len([key for key,value1,value2,image in all_list_array if key == i])
  count = len_label//13
  if count >1:
    kmeans2_label1 = KMeans(n_clusters=count ,random_state =0)

    result = [value2 for key,value1,value2,image in all_list_array if key == i]
    result = np.array(result)

    kmeans2_label1.fit(result)

    print('Centers found by scikit-learn:')
    print(kmeans2.cluster_centers_)
    result_pred_label2 = (kmeans2_label1.predict(result))
    # kmeans_display(result, result_pred_label2)
    for index, (key1, key2, image, value1,value2) in enumerate(all_list_image):
      for i in range(len(result_pred_label2)):
          if (value2 == result[i]).all():
              # Tạo một bản sao của tuple hiện tại
              updated_tuple = list(all_list_image[index])
              # Thay đổi giá trị của key2 trong danh sách
              updated_tuple[1] = result_pred_label2[i]
              # Chuyển đổi danh sách đã được cập nhật thành tuple và gán lại vào all_list_image
              all_list_image[index] = tuple(updated_tuple)

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set

Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 23.99286786  97.65680714 191.39643571]
 [ 94.85196     94.31786    107.31125333]
 [ 20.26515333 172.23953333 173.90971333]
 [ 21.74945769  69.93094231 195.87871154]
 [ 46.78221935  55.6506129  155.11283548]
 [ 39.788485   127.585285   100.630485  ]
 [ 27.44935714  26.31848571 179.77884286]
 [ 17.480295    53.495255   222.478025  ]
 [ 22.03905926  69.8302037  144.14539259]
 [ 21.75761429  96.81638571 164.37935714]]
Centers found by scikit-learn:
[[ 74.22933333  15.32813333 165.43575333]
 [ 23.99286786  97.65680714 191.39643571]
 [ 94.85196     94.31786    107.31125333]
 [ 20.26515333 172.23953333 173.90971333]
 [ 21.74945769  69.93094231 195.87871154]
 [ 46.78221935  55.6506129  155.11283548]
 [ 39.788485   127.585285   100.630485  ]
 [ 27.44935714  26.31848571 179.77884286]
 [ 17.480295    53.495255   222.478025  ]
 [ 22.03905926  69.8302037  144.14539259]
 [ 21.75761429  96.81638571 164.37935714]]
Centers foun

In [17]:
tmp = [[] for _ in range(11)]
for key1, key2, image, value1, value2 in all_list_image:
  if key2 != None:
    tmp[key1].append([key2,image,value1,value2])

for index, (i) in enumerate(tmp):
  if len(i)>0:
    print(index)
    for key2,image,value1,value2 in i:
      if key2==0:
        print(image)
    print()
    for key2,image,value1,value2 in i:
      if key2==1:
        print(image)

1
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Banana (14).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (01).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (05).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (11).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (12).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Kiwi (13).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (03).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (04).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (05).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (06).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (07).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (08).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (09).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (10).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (11).jpg
D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix\Lemon (12).jpg
D:\PTIT\CS